Our task is to make a simple DL classifer to correctly classify frauds.

In [0]:
# Starting with useful imports
import csv
import numpy as np
import matplotlib.pyplot as plt

In [0]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive
/gdrive


In [0]:
%ls

'My Drive'/


In [0]:
fname = 'My Drive/data/creditcard.csv'
all_features = []
all_targets = []
with open(fname) as f:
  for i, line in enumerate(f):
    if i == 0:
      print('HEADER:', line.strip())
      continue  # Skip header
    fields = line.strip().split(',')
    all_features.append([float(v.replace('"', '')) for v in fields[:-1]])
    all_targets.append([int(fields[-1].replace('"', ''))])
    if i == 1:
      print('EXAMPLE FEATURES:', all_features[-1])
    
features = np.array(all_features, dtype='float32')
targets = np.array(all_targets, dtype='uint8')
print('features.shape:', features.shape)
print('targets.shape:', targets.shape)

HEADER: "Time","V1","V2","V3","V4","V5","V6","V7","V8","V9","V10","V11","V12","V13","V14","V15","V16","V17","V18","V19","V20","V21","V22","V23","V24","V25","V26","V27","V28","Amount","Class"
EXAMPLE FEATURES: [0.0, -1.3598071336738, -0.0727811733098497, 2.53634673796914, 1.37815522427443, -0.338320769942518, 0.462387777762292, 0.239598554061257, 0.0986979012610507, 0.363786969611213, 0.0907941719789316, -0.551599533260813, -0.617800855762348, -0.991389847235408, -0.311169353699879, 1.46817697209427, -0.470400525259478, 0.207971241929242, 0.0257905801985591, 0.403992960255733, 0.251412098239705, -0.018306777944153, 0.277837575558899, -0.110473910188767, 0.0669280749146731, 0.128539358273528, -0.189114843888824, 0.133558376740387, -0.0210530534538215, 149.62]
features.shape: (284807, 30)
targets.shape: (284807, 1)


In [0]:
num_test_samples = int(len(features) * 0.2)
train_features = features[:-num_test_samples]
train_targets = targets[:-num_test_samples]
test_features = features[-num_test_samples:]
test_targets = targets[-num_test_samples:]
print('Number of training samples:', len(train_features))
print('Number of test samples:', len(test_features))

Number of training samples: 227846
Number of test samples: 56961


In [0]:
counts = np.bincount(train_targets[:, 0])
print('Number of positive samples in training data: {} ({:.2f}% of total)'.format(counts[1], 100 * float(counts[1]) / len(train_targets)))

Number of positive samples in training data: 417 (0.18% of total)


In [0]:
counts = np.bincount(test_targets[:, 0])
print('Number of positive samples in testing data: {} ({:.2f}% of total)'.format(counts[1], 100 * float(counts[1]) / len(test_targets)))

Number of positive samples in testing data: 75 (0.13% of total)


In [0]:
mean = np.mean(train_features, axis=0)
train_features -= mean
test_features -= mean
std = np.std(train_features, axis=0)
train_features /= std
test_features /= std

In [0]:
train_features.shape

(227846, 30)

In [0]:
from tensorflow import keras
import tensorflow as tf

session = keras.backend.get_session()
init = tf.global_variables_initializer()
session.run(init)

model = keras.Sequential([
  keras.layers.Dense(1024, activation='relu',
                     input_shape=(train_features.shape[-1],)),
  keras.layers.Dense(512, activation='tanh'),
  keras.layers.Dropout(0.25),
  keras.layers.Dense(256, activation='relu'),
  keras.layers.Dropout(0.40),
  keras.layers.Dense(128, activation='elu'),
  keras.layers.Dropout(0.40),
  keras.layers.Dense(1, activation='sigmoid'),
])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 1024)              31744     
_________________________________________________________________
dense_6 (Dense)              (None, 512)               524800    
_________________________________________________________________
dropout_3 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 256)               131328    
_________________________________________________________________
dropout_4 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 128)               32896     
_________________________________________________________________
dropout_5 (Dropout)          (None, 128)              

In [0]:
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

Using TensorFlow backend.


['/job:localhost/replica:0/task:0/device:GPU:0']

In [0]:
metrics = [keras.metrics.FalseNegatives(name='fn'),
           keras.metrics.FalsePositives(name='fp'),
           keras.metrics.TrueNegatives(name='tn'),
           keras.metrics.TruePositives(name='tp'),
           keras.metrics.Precision(name='precision'),
           keras.metrics.Recall(name='recall'),
           keras.metrics.Accuracy(name='accuracy')]

model.compile(optimizer=keras.optimizers.Adam(1e-2),
              loss='binary_crossentropy',
              metrics=metrics)

#callbacks = [keras.callbacks.ModelCheckpoint('fraud_model_at_epoch_{epoch}.h5')]
class_weight = {0: 1, 1: 200}

model.fit(train_features, train_targets,
          batch_size=4096, epochs=100, verbose=2, validation_data=(test_features, test_targets), class_weight=class_weight)

Train on 227846 samples, validate on 56961 samples
Epoch 1/100
227846/227846 - 1s - loss: 0.5196 - fn: 71.0000 - fp: 7932.0000 - tn: 219497.0000 - tp: 346.0000 - precision: 0.0418 - recall: 0.8297 - accuracy: 0.0813 - val_loss: 0.0153 - val_fn: 14.0000 - val_fp: 85.0000 - val_tn: 56801.0000 - val_tp: 61.0000 - val_precision: 0.4178 - val_recall: 0.8133 - val_accuracy: 0.0000e+00
Epoch 2/100
227846/227846 - 1s - loss: 0.1740 - fn: 45.0000 - fp: 2758.0000 - tn: 224671.0000 - tp: 372.0000 - precision: 0.1188 - recall: 0.8921 - accuracy: 0.0266 - val_loss: 0.0164 - val_fn: 15.0000 - val_fp: 30.0000 - val_tn: 56856.0000 - val_tp: 60.0000 - val_precision: 0.6667 - val_recall: 0.8000 - val_accuracy: 0.0000e+00
Epoch 3/100
227846/227846 - 1s - loss: 0.1557 - fn: 36.0000 - fp: 2657.0000 - tn: 224772.0000 - tp: 381.0000 - precision: 0.1254 - recall: 0.9137 - accuracy: 0.0261 - val_loss: 0.0082 - val_fn: 13.0000 - val_fp: 72.0000 - val_tn: 56814.0000 - val_tp: 62.0000 - val_precision: 0.4627 - va